In [1]:
import pandas as pd
import re
import jieba
#构建中文分词类v3.0
class Leijieba:    
    
    __stopwords = ''
        
    def __init__(self,s,t='',b=''):    #t为停用词的文件参数（路径+文件名）；b为本地词库的文件参数（路径+文件名）
        self.__s = s
        
        if t != '':
            self.__stopwords = [line.strip() for line in open(t,encoding='UTF-8').readlines()]  #逐行读取停用词文件，放入类内部变量__stopwords
            
        if b != '':
            jieba.load_userdict(b)
        
    def cut(self):
        seg_list = jieba.cut(self.__s, cut_all=False)
        if self.__stopwords != '':
            outstr = []    #目的是存储去掉停用用词的分词结果
            for word in list(seg_list):
                if word not in self.__stopwords:
                    if word.strip() !='':   #去空格
                        outstr.append(word)
                        
            seg_list = outstr                                                
        return list(seg_list)
    
    def num(self):
        lst = self.cut()
        
        word_dict = {}        
        for item in lst:
            if item not in word_dict:
                word_dict[item] = 1   #新建“键-值”
            else:
                word_dict[item] += 1  #增加 键的值
        return sorted(word_dict.items(), key=lambda e:e[1], reverse=True)  
    
    def cut1(self,num = 0,sw = 0):   #1.删除词频低于lnum的词,2.if sw ==1,删除单字的词
        lst = self.cut()
        
        word_dict = {}        
        for item in lst:
            if item not in word_dict:
                word_dict[item] = 1   #新建“键-值”
            else:
                word_dict[item] += 1  #增加 键的值
        dic = dict(sorted(word_dict.items(), key=lambda e:e[1], reverse=True)) 
        lst1 = lst[:]
        
        if sw == 1:
            for item in lst:
                if len(item) == 1:
                    lst1.remove(item)
                else:
                    if dic[item] <= num:
                        lst1.remove(item)
        else:
            for item in lst:
                if dic[item] <= num:
                    lst1.remove(item)
                
        return lst1

In [3]:
df_news = pd.read_csv('./预处理过的3000条论坛数据.csv',\
                        usecols=['summarization','x'], encoding='utf-8')
df_news.head()

,summarization,x
0,金价格在亚洲盘时段上行趋势线后遭遇强力阻力收出中等长度的下影线看涨的因素还在待到债务危机避险...,1
1,从日K线来看黄金处于交易日大幅走低跌破5日均线后大幅回落至200日平均线即115附近获得支撑,1
2,黄金每克价格太贵了没人买得起房子一样贵了工资十年没见涨落都在1000元左右一月下来也买不了多...,0
3,捕鱼收网现货金多头在1230美元上方获利平仓美国制造业数据降避险情绪继续升但是美元与黄金分道...,1
4,这里是不是做黄金的比较多小弟有意向往这方面发展可惜不太了解希望老手们有空多多指教,1


In [5]:
s = 'AAA我是分词的间隔符BBB'.join(list(df_news['summarization']))

In [7]:
m = Leijieba(s,t='./stopwords.txt',\
             b='./userdic.txt')
s2 = ' '.join(m.cut1(10,1))
txtf_news = open('./cnews.jieba1.txt','wt', encoding='utf-8')
txtf_news.write(s2.replace('AAA我是分词的间隔符BBB','\n'))
txtf_news.close()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\86155\AppData\Local\Temp\jieba.cache
Loading model cost 0.574 seconds.
Prefix dict has been built successfully.


In [8]:
txt = pd.read_csv('./cnews.jieba1.txt',\
                  skip_blank_lines=False,encoding = 'utf-8', header = None)
outer_join = df_news.join(txt, how='outer')
outer_join.columns = ['summarization', 'x', 'x']

In [11]:
outer_join

,summarization,x,x
0,金价格在亚洲盘时段上行趋势线后遭遇强力阻力收出中等长度的下影线看涨的因素还在待到债务危机避险...,1,价格 亚洲 时段 上行 趋势 阻力 因素 债务 危机 避险 情绪 经济 放缓 美国 经济 影...
1,从日K线来看黄金处于交易日大幅走低跌破5日均线后大幅回落至200日平均线即115附近获得支撑,1,黄金 处于 交易日 大幅 走低 跌破 均线 大幅 回落 支撑
2,黄金每克价格太贵了没人买得起房子一样贵了工资十年没见涨落都在1000元左右一月下来也买不了多...,0,黄金 价格 市场 赚钱
3,捕鱼收网现货金多头在1230美元上方获利平仓美国制造业数据降避险情绪继续升但是美元与黄金分道...,1,现货 多头 美元 上方 获利 平仓 美国 数据 避险 情绪 美元 黄金 周一 最低 下跌 ...
4,这里是不是做黄金的比较多小弟有意向往这方面发展可惜不太了解希望老手们有空多多指教,1,黄金 发展
...,...,...,...
2030,一个骗局让许多人梦碎黄金,0,黄金
2031,老美扭曲购买债券计划也是拖延时间而已美元的宿命并没改变欧洲股市跌得厉害有机构卖黄金白银对冲黄...,1,计划 美元 改变 欧洲 股市 机构 黄金 白银 对冲 黄金 改变
2032,一旦对经济又有期望股市见底后全球资金又将反抢已经惨跌的股市很可能雪上加霜黄金被彻底抛弃,0,经济 股市 全球 资金 股市 黄金
2033,很遗憾我是愁的那一个只能自我安慰下浅入浅出总比满满爆仓的好黄金已经疯狂的没有支撑没有底位了只有跌,1,黄金 疯狂


In [13]:
outer_join.columns = ['summarization', 'category', 'x']
outer_join

,summarization,category,x
0,金价格在亚洲盘时段上行趋势线后遭遇强力阻力收出中等长度的下影线看涨的因素还在待到债务危机避险...,1,价格 亚洲 时段 上行 趋势 阻力 因素 债务 危机 避险 情绪 经济 放缓 美国 经济 影...
1,从日K线来看黄金处于交易日大幅走低跌破5日均线后大幅回落至200日平均线即115附近获得支撑,1,黄金 处于 交易日 大幅 走低 跌破 均线 大幅 回落 支撑
2,黄金每克价格太贵了没人买得起房子一样贵了工资十年没见涨落都在1000元左右一月下来也买不了多...,0,黄金 价格 市场 赚钱
3,捕鱼收网现货金多头在1230美元上方获利平仓美国制造业数据降避险情绪继续升但是美元与黄金分道...,1,现货 多头 美元 上方 获利 平仓 美国 数据 避险 情绪 美元 黄金 周一 最低 下跌 ...
4,这里是不是做黄金的比较多小弟有意向往这方面发展可惜不太了解希望老手们有空多多指教,1,黄金 发展
...,...,...,...
2030,一个骗局让许多人梦碎黄金,0,黄金
2031,老美扭曲购买债券计划也是拖延时间而已美元的宿命并没改变欧洲股市跌得厉害有机构卖黄金白银对冲黄...,1,计划 美元 改变 欧洲 股市 机构 黄金 白银 对冲 黄金 改变
2032,一旦对经济又有期望股市见底后全球资金又将反抢已经惨跌的股市很可能雪上加霜黄金被彻底抛弃,0,经济 股市 全球 资金 股市 黄金
2033,很遗憾我是愁的那一个只能自我安慰下浅入浅出总比满满爆仓的好黄金已经疯狂的没有支撑没有底位了只有跌,1,黄金 疯狂


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word', max_features=4000,  lowercase = False)
vectorizer.fit(list(outer_join['x']))
X = vectorizer.transform(list(outer_join['x']))

In [16]:
X.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.29028051,
        0.        ]])

In [19]:
import numpy as np
ordering = [0, 1]
y = np.array(outer_join['category'].map(lambda x: ordering.index(x)))

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [23]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

MultinomialNB()

In [25]:
y_pred = classifier.predict(X_test)

In [27]:
from sklearn.metrics import classification_report
target_names = [str(i) for i in ordering]
t = classification_report(y_test, y_pred, target_names=target_names)
print(t)

              precision    recall  f1-score   support

           0       0.84      0.66      0.74       112
           1       0.91      0.96      0.94       397

    accuracy                           0.90       509
   macro avg       0.88      0.81      0.84       509
weighted avg       0.89      0.90      0.89       509



####预测一万多条

In [30]:
outer_join_1 = pd.read_csv('./分词过的.csv')
outer_join_1

,publish_time,view_counts\t,summarization,x
0,2011-12-19 14:32:05,589,对于天通黄金的操作空间留有余地我提出了一些建议和建议。,天通 黄金 操作 空间 提出 建议 建议
1,2011-12-19 14:38:52,270,天津贵金属交易所推出的实物黄金投资金条延迟交收交易天通金 现货白银 主要优势,天津 贵金属 交易所 推出 实物 黄金 投资 金条 交易 天通金 现货 白银 优势
2,2011-12-19 14:40:57,681,"国际黄金市场行情星期一(7月16日)凌晨3 00 分开盘,交易佣金为万分之八",国际 黄金市场 行情 星期一 凌晨 00 开盘 交易
3,2011-12-19 15:05:09,20303,黄金白银最新技术分析继续看空黄金任然处于下跌的趋势。,黄金 白银 最新 技术 分析 看空 黄金 处于 下跌 趋势
4,2011-12-19 15:25:40,655,周四(1月1日)黄金周线图跌势已缓,周四 黄金周 线图 跌势
...,...,...,...,...
110284,2024-08-09 23:39:20,101,白银暴跌黄金硬撑白银反弹黄金暴涨资本根据各自散户的持仓基数分别收割,白银 暴跌 黄金 白银 反弹 黄金 暴涨 持仓
110285,2024-08-10 00:42:22,85,都说黄金只能多不能空 不可能下跌黄金才是真正做多赚钱养家糊口,黄金 只能 下跌 黄金 赚钱
110286,2024-08-10 02:23:27,125,黄 金 现 价 2 4 2 8 2 7 附 近 位 可 以 适 当 的 做 多 上 来 美原...,原油 外汇 黄金 国际 贵金属
110287,2024-08-10 15:14:49,313,金价贵吗全球黄金市值16万亿美元加密货币市值都2 6万亿美元虚拟的不到10年就这么高市值黄金...,金价 全球 黄金 市值 万亿美元 货币 市值 万亿美元 不到 市值 黄金


In [32]:
X_1 = vectorizer.transform(list(outer_join_1['x']))

In [34]:
X_1.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.3986654 , 0.        , 0.        , ..., 0.        , 0.        ,
        0.36078595],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [36]:
y_pred_1 = classifier.predict(X_1)
print(y_pred_1)

[0 0 0 ... 1 1 0]


In [38]:
outer_join_1['相关性'] = 2

In [40]:
outer_join_1

,publish_time,view_counts\t,summarization,x,相关性
0,2011-12-19 14:32:05,589,对于天通黄金的操作空间留有余地我提出了一些建议和建议。,天通 黄金 操作 空间 提出 建议 建议,2
1,2011-12-19 14:38:52,270,天津贵金属交易所推出的实物黄金投资金条延迟交收交易天通金 现货白银 主要优势,天津 贵金属 交易所 推出 实物 黄金 投资 金条 交易 天通金 现货 白银 优势,2
2,2011-12-19 14:40:57,681,"国际黄金市场行情星期一(7月16日)凌晨3 00 分开盘,交易佣金为万分之八",国际 黄金市场 行情 星期一 凌晨 00 开盘 交易,2
3,2011-12-19 15:05:09,20303,黄金白银最新技术分析继续看空黄金任然处于下跌的趋势。,黄金 白银 最新 技术 分析 看空 黄金 处于 下跌 趋势,2
4,2011-12-19 15:25:40,655,周四(1月1日)黄金周线图跌势已缓,周四 黄金周 线图 跌势,2
...,...,...,...,...,...
110284,2024-08-09 23:39:20,101,白银暴跌黄金硬撑白银反弹黄金暴涨资本根据各自散户的持仓基数分别收割,白银 暴跌 黄金 白银 反弹 黄金 暴涨 持仓,2
110285,2024-08-10 00:42:22,85,都说黄金只能多不能空 不可能下跌黄金才是真正做多赚钱养家糊口,黄金 只能 下跌 黄金 赚钱,2
110286,2024-08-10 02:23:27,125,黄 金 现 价 2 4 2 8 2 7 附 近 位 可 以 适 当 的 做 多 上 来 美原...,原油 外汇 黄金 国际 贵金属,2
110287,2024-08-10 15:14:49,313,金价贵吗全球黄金市值16万亿美元加密货币市值都2 6万亿美元虚拟的不到10年就这么高市值黄金...,金价 全球 黄金 市值 万亿美元 货币 市值 万亿美元 不到 市值 黄金,2


In [48]:
for i in range(0,110289):
    outer_join_1.loc[i,'相关性'] = y_pred_1[i]

,summarization,category,x
0,金价格在亚洲盘时段上行趋势线后遭遇强力阻力收出中等长度的下影线看涨的因素还在待到债务危机避险...,1,价格 亚洲 时段 上行 趋势 阻力 因素 债务 危机 避险 情绪 经济 放缓 美国 经济 影...
1,从日K线来看黄金处于交易日大幅走低跌破5日均线后大幅回落至200日平均线即115附近获得支撑,1,黄金 处于 交易日 大幅 走低 跌破 均线 大幅 回落 支撑
2,黄金每克价格太贵了没人买得起房子一样贵了工资十年没见涨落都在1000元左右一月下来也买不了多...,0,黄金 价格 市场 赚钱
3,捕鱼收网现货金多头在1230美元上方获利平仓美国制造业数据降避险情绪继续升但是美元与黄金分道...,1,现货 多头 美元 上方 获利 平仓 美国 数据 避险 情绪 美元 黄金 周一 最低 下跌 ...
4,这里是不是做黄金的比较多小弟有意向往这方面发展可惜不太了解希望老手们有空多多指教,1,黄金 发展
...,...,...,...
2030,一个骗局让许多人梦碎黄金,0,黄金
2031,老美扭曲购买债券计划也是拖延时间而已美元的宿命并没改变欧洲股市跌得厉害有机构卖黄金白银对冲黄...,1,计划 美元 改变 欧洲 股市 机构 黄金 白银 对冲 黄金 改变
2032,一旦对经济又有期望股市见底后全球资金又将反抢已经惨跌的股市很可能雪上加霜黄金被彻底抛弃,0,经济 股市 全球 资金 股市 黄金
2033,很遗憾我是愁的那一个只能自我安慰下浅入浅出总比满满爆仓的好黄金已经疯狂的没有支撑没有底位了只有跌,1,黄金 疯狂


In [50]:
outer_join_1

,publish_time,view_counts\t,summarization,x,相关性
0,2011-12-19 14:32:05,589,对于天通黄金的操作空间留有余地我提出了一些建议和建议。,天通 黄金 操作 空间 提出 建议 建议,0
1,2011-12-19 14:38:52,270,天津贵金属交易所推出的实物黄金投资金条延迟交收交易天通金 现货白银 主要优势,天津 贵金属 交易所 推出 实物 黄金 投资 金条 交易 天通金 现货 白银 优势,0
2,2011-12-19 14:40:57,681,"国际黄金市场行情星期一(7月16日)凌晨3 00 分开盘,交易佣金为万分之八",国际 黄金市场 行情 星期一 凌晨 00 开盘 交易,0
3,2011-12-19 15:05:09,20303,黄金白银最新技术分析继续看空黄金任然处于下跌的趋势。,黄金 白银 最新 技术 分析 看空 黄金 处于 下跌 趋势,1
4,2011-12-19 15:25:40,655,周四(1月1日)黄金周线图跌势已缓,周四 黄金周 线图 跌势,1
...,...,...,...,...,...
110284,2024-08-09 23:39:20,101,白银暴跌黄金硬撑白银反弹黄金暴涨资本根据各自散户的持仓基数分别收割,白银 暴跌 黄金 白银 反弹 黄金 暴涨 持仓,1
110285,2024-08-10 00:42:22,85,都说黄金只能多不能空 不可能下跌黄金才是真正做多赚钱养家糊口,黄金 只能 下跌 黄金 赚钱,1
110286,2024-08-10 02:23:27,125,黄 金 现 价 2 4 2 8 2 7 附 近 位 可 以 适 当 的 做 多 上 来 美原...,原油 外汇 黄金 国际 贵金属,1
110287,2024-08-10 15:14:49,313,金价贵吗全球黄金市值16万亿美元加密货币市值都2 6万亿美元虚拟的不到10年就这么高市值黄金...,金价 全球 黄金 市值 万亿美元 货币 市值 万亿美元 不到 市值 黄金,1


In [52]:
data_cleaned = outer_join_1.loc[outer_join_1['相关性'] != 0]
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103200 entries, 3 to 110287
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   publish_time   103200 non-null  object
 1   view_counts	   103200 non-null  object
 2   summarization  103200 non-null  object
 3   x              103200 non-null  object
 4   相关性            103200 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 4.7+ MB


In [56]:
data_cleaned = data_cleaned.drop(['x', '相关性'], axis=1)
data_cleaned = data_cleaned.rename(columns={'view_counts\t': 'view_counts'})
data_cleaned = data_cleaned.reset_index(drop=True)
data_cleaned

,publish_time,view_counts,summarization
0,2011-12-19 15:05:09,20303,黄金白银最新技术分析继续看空黄金任然处于下跌的趋势。
1,2011-12-19 15:25:40,655,周四(1月1日)黄金周线图跌势已缓
2,2011-12-20 03:53:48,1547,想慢慢成才听太公的想快速致富听一亿的我是相信一亿在几天前说1610反弹到位观点果然昨晚到了十...
3,2011-12-20 07:19:40,1330,黄金从400到320跌幅20 白银纸从10块跌到6快跌幅40 大家都以为白银要到8块包括我在...
4,2011-12-20 07:43:38,925,银品金屋12月20日早间黄金行情分析分析
...,...,...,...
103195,2024-08-09 23:19:06,94,机 会 又 来 了 想 做 单 的 同 学 金 子 现 价 2 4 2 7 2 6 附 近 ...
103196,2024-08-09 23:39:20,101,白银暴跌黄金硬撑白银反弹黄金暴涨资本根据各自散户的持仓基数分别收割
103197,2024-08-10 00:42:22,85,都说黄金只能多不能空 不可能下跌黄金才是真正做多赚钱养家糊口
103198,2024-08-10 02:23:27,125,黄 金 现 价 2 4 2 8 2 7 附 近 位 可 以 适 当 的 做 多 上 来 美原...


In [58]:
data_cleaned.to_csv('./朴素贝叶斯处理过的第二部分数据.csv',index = False)

###预测新的数据

In [31]:
df_news_1 = pd.read_csv('./预处理过的1500条论坛数据.csv',\
                        usecols=['summarization'], encoding='utf-8')
df_news_1.head()

,summarization
0,老秦谈金黄金操作十大手法
1,老秦谈金黄金交易如何保障单子隔夜利润
2,受股市暴跌拖累隔夜国际金价也出现暴下跌最低下跌至1720美元
3,思维转换之快终于在黄金身上体现的淋漓尽至上2000的梦还没做醒马上又要被熊来了的观点占拒淡定...
4,美国经济周期研究所周五(9月16日)称美国上调黄金白银期货交易日的领先指标小幅回落至122 ...


In [35]:
s_1 = 'AAA我是分词的间隔符BBB'.join(list(df_news_1['summarization']))
m_1 = Leijieba(s,t='./stopwords.txt',\
             b='./userdic.txt')
s2_1 = ' '.join(m.cut1(10,1))
txtf_news_1 = open('./cnews.jieba1.txt','wt', encoding='utf-8')
txtf_news_1.write(s2.replace('AAA我是分词的间隔符BBB','\n'))
txtf_news_1.close()
txt_1 = pd.read_csv('./cnews.jieba1.txt',\
                  skip_blank_lines=False,encoding = 'utf-8', header = None)
outer_join_1 = df_news_1.join(txt, how='outer')
outer_join_1.columns = ['summarization', 'x']

In [37]:
outer_join_1

,summarization,x
0,老秦谈金黄金操作十大手法,黄金 操作
1,老秦谈金黄金交易如何保障单子隔夜利润,黄金 交易
2,受股市暴跌拖累隔夜国际金价也出现暴下跌最低下跌至1720美元,股市 暴跌 国际 金价 下跌 最低 下跌 1720 美元
3,思维转换之快终于在黄金身上体现的淋漓尽至上2000的梦还没做醒马上又要被熊来了的观点占拒淡定...,黄金 交易日 市场 黄金 上涨
4,美国经济周期研究所周五(9月16日)称美国上调黄金白银期货交易日的领先指标小幅回落至122 ...,美国 周五 美国 黄金 白银 小幅 回落
...,...,...
955,金市周五(12月19日)公布的最新单日数据显示金价自上周五以来首次触及80至80位并一度跌破...,金市 周五 公布 最新 单日 数据 显示 金价 上周五 首次 触及 一度 跌破 16000
956,美国天通金周一(2月3日)连续两个交易日下滑主要受美元走软和空头回补的压力 但金价本周仍可能...,美国 天通金 周一 连续 两个 交易日 下滑 美元 空头 压力 金价 本周 创下 跌幅
957,天通金投资优势 1T0交易每日可以交易数次 增加获利机会 弥补国际黄金报价波动最频繁的时间段,天通金 投资 交易 交易 机会 国际 黄金 波动
958,黄金是亘古不变的事实。,黄金


In [39]:
X_1 = vectorizer.transform(list(outer_join_1['x']))

In [41]:
X_1.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [43]:
y_pred_1 = classifier.predict(X_1)
print(y_pred_1)

[0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
 0 1 1 1 0 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 0 1 0 1 1 0 0 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1 1 1 1 0 1 0 1 0 1 0
 0 1 1 1 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 0 0 0 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1
 0 0 1 0 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1
 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1
 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 1 0 0 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 0 0 0
 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 0 1 1 1 1 1 0 1 1
 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 1 1 1 1 0 1 0 1 1 1 

In [49]:
df_news_1['相关性'] = 2

In [51]:
df_news_1

,summarization,相关性
0,老秦谈金黄金操作十大手法,2
1,老秦谈金黄金交易如何保障单子隔夜利润,2
2,受股市暴跌拖累隔夜国际金价也出现暴下跌最低下跌至1720美元,2
3,思维转换之快终于在黄金身上体现的淋漓尽至上2000的梦还没做醒马上又要被熊来了的观点占拒淡定...,2
4,美国经济周期研究所周五(9月16日)称美国上调黄金白银期货交易日的领先指标小幅回落至122 ...,2
...,...,...
955,金市周五(12月19日)公布的最新单日数据显示金价自上周五以来首次触及80至80位并一度跌破...,2
956,美国天通金周一(2月3日)连续两个交易日下滑主要受美元走软和空头回补的压力 但金价本周仍可能...,2
957,天通金投资优势 1T0交易每日可以交易数次 增加获利机会 弥补国际黄金报价波动最频繁的时间段,2
958,黄金是亘古不变的事实。,2


In [53]:
for i in range(0,960):
    df_news_1.loc[i,'相关性'] = y_pred_1[i]

In [69]:
df_news_1

,summarization,相关性
0,老秦谈金黄金操作十大手法,0
1,老秦谈金黄金交易如何保障单子隔夜利润,0
2,受股市暴跌拖累隔夜国际金价也出现暴下跌最低下跌至1720美元,1
3,思维转换之快终于在黄金身上体现的淋漓尽至上2000的梦还没做醒马上又要被熊来了的观点占拒淡定...,1
4,美国经济周期研究所周五(9月16日)称美国上调黄金白银期货交易日的领先指标小幅回落至122 ...,1
...,...,...
955,金市周五(12月19日)公布的最新单日数据显示金价自上周五以来首次触及80至80位并一度跌破...,1
956,美国天通金周一(2月3日)连续两个交易日下滑主要受美元走软和空头回补的压力 但金价本周仍可能...,1
957,天通金投资优势 1T0交易每日可以交易数次 增加获利机会 弥补国际黄金报价波动最频繁的时间段,0
958,黄金是亘古不变的事实。,1


In [71]:
data_cleaned = df_news_1.loc[df_news_1['相关性'] != 0]
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 792 entries, 2 to 959
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   summarization  792 non-null    object
 1   相关性            792 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 18.6+ KB


In [73]:
data_cleaned

,summarization,相关性
2,受股市暴跌拖累隔夜国际金价也出现暴下跌最低下跌至1720美元,1
3,思维转换之快终于在黄金身上体现的淋漓尽至上2000的梦还没做醒马上又要被熊来了的观点占拒淡定...,1
4,美国经济周期研究所周五(9月16日)称美国上调黄金白银期货交易日的领先指标小幅回落至122 ...,1
5,9月23日纽约商品交易所黄金期货价格12月合约周五每盎司暴跌101,1
6,话虽偏颇了些但比之股市和大宗商品动者几倍甚致十几倍的涨幅相比黄金明显是落后了我们只是从去年才...,1
...,...,...
954,国际黄金周五如期出现反弹但由于惠誉下调欧洲五大商业银行评级使得欧债担忧加剧资金加速流入美元预...,1
955,金市周五(12月19日)公布的最新单日数据显示金价自上周五以来首次触及80至80位并一度跌破...,1
956,美国天通金周一(2月3日)连续两个交易日下滑主要受美元走软和空头回补的压力 但金价本周仍可能...,1
958,黄金是亘古不变的事实。,1


In [75]:
data_cleaned = data_cleaned['summarization']
data_cleaned = data_cleaned.reset_index(drop=True)
data_cleaned

0                         受股市暴跌拖累隔夜国际金价也出现暴下跌最低下跌至1720美元
1      思维转换之快终于在黄金身上体现的淋漓尽至上2000的梦还没做醒马上又要被熊来了的观点占拒淡定...
2      美国经济周期研究所周五(9月16日)称美国上调黄金白银期货交易日的领先指标小幅回落至122 ...
3                      9月23日纽约商品交易所黄金期货价格12月合约周五每盎司暴跌101
4      话虽偏颇了些但比之股市和大宗商品动者几倍甚致十几倍的涨幅相比黄金明显是落后了我们只是从去年才...
                             ...                        
787    国际黄金周五如期出现反弹但由于惠誉下调欧洲五大商业银行评级使得欧债担忧加剧资金加速流入美元预...
788    金市周五(12月19日)公布的最新单日数据显示金价自上周五以来首次触及80至80位并一度跌破...
789    美国天通金周一(2月3日)连续两个交易日下滑主要受美元走软和空头回补的压力 但金价本周仍可能...
790                                          黄金是亘古不变的事实。
791    上周五欧洲国家主权评级机构惠誉穆迪相继下调欧元兑美元走低美元指数小幅上涨周一亚欧盘金价止跌反...
Name: summarization, Length: 792, dtype: object

In [77]:
data_cleaned.to_csv('./预处理过的792条论坛数据.csv',index=False)